In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!nvidia-smi

Wed Oct 19 16:13:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    47W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!cp -av '/content/drive/MyDrive/coco/pycocotools' '/content/'

'/content/drive/MyDrive/coco/pycocotools' -> '/content/pycocotools'
'/content/drive/MyDrive/coco/pycocotools/_mask.pyx' -> '/content/pycocotools/_mask.pyx'
'/content/drive/MyDrive/coco/pycocotools/__init__.py' -> '/content/pycocotools/__init__.py'
'/content/drive/MyDrive/coco/pycocotools/cocoeval.py' -> '/content/pycocotools/cocoeval.py'
'/content/drive/MyDrive/coco/pycocotools/_mask.c' -> '/content/pycocotools/_mask.c'
'/content/drive/MyDrive/coco/pycocotools/_mask.cp39-win_amd64.pyd' -> '/content/pycocotools/_mask.cp39-win_amd64.pyd'
'/content/drive/MyDrive/coco/pycocotools/mask.py' -> '/content/pycocotools/mask.py'
'/content/drive/MyDrive/coco/pycocotools/coco.py' -> '/content/pycocotools/coco.py'
'/content/drive/MyDrive/coco/pycocotools/__pycache__' -> '/content/pycocotools/__pycache__'
'/content/drive/MyDrive/coco/pycocotools/__pycache__/coco.cpython-39.pyc' -> '/content/pycocotools/__pycache__/coco.cpython-39.pyc'
'/content/drive/MyDrive/coco/pycocotools/__pycache__/__init__.cpyt

In [5]:
# Thêm thư viện
from pycocotools.coco import COCO

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
# from keras.layers.wrappers import Bidirectional
# from keras.layers.merge import add
from keras.layers import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tqdm.notebook import tqdm
import time
from random import shuffle
from tensorflow.keras.utils import load_img, img_to_array 

In [6]:
#1. train_imgs
annFile = "/content/drive/MyDrive/coco/annotations/captions_train2014.json"
coco = COCO(annFile)
imgIds = coco.getImgIds()
print('imgIds ',len(imgIds))

shuffle(imgIds)

n = 50000
m = n+500
train_ids = imgIds[:n]
test_ids = imgIds[n:m]

print('train_ids',train_ids[:5])
print('test_ids',test_ids[:5])

# get imgs name
train_imgs = []
test_imgs = []

images = '/content/drive/MyDrive/coco/train2014/'

for i in range(len(train_ids)):
    train_imgs.append(images+ 'COCO_train2014_' + '%012d' % (train_ids[i]) + '.jpg')

for i in range(len(test_ids)):
    test_imgs.append(images+'COCO_train2014_' + '%012d' % (test_ids[i]) + '.jpg')


print('len(train_imgs)',len(train_imgs)) #50000
print(train_imgs[:5])
print(test_imgs[:5])

file = open('/content/drive/MyDrive/Inception_LSTM_coco/training.txt',"w")
file.write(str(train_imgs))
file.close()

file = open('/content/drive/MyDrive/Inception_LSTM_coco/testing.txt',"w")
file.write(str(test_imgs))
file.close()

loading annotations into memory...
Done (t=1.80s)
creating index...
index created!
imgIds  82783
train_ids [63943, 562740, 503606, 348042, 125908]
test_ids [359147, 168161, 294323, 181906, 87850]
len(train_imgs) 50000
['/content/drive/MyDrive/coco/train2014/COCO_train2014_000000063943.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000562740.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000503606.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000348042.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000125908.jpg']
['/content/drive/MyDrive/coco/train2014/COCO_train2014_000000359147.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000168161.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000294323.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000181906.jpg', '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000087850.jpg']


In [11]:
#2.train_features
# Load ảnh, resize về kích thước mà Inception v3 yêu cầu.
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

# Load the inception v3 model
model = InceptionV3(weights='imagenet')
# # Tạo model mới, bỏ layer cuối từ inception v3
model_new = Model(model.input, model.layers[-2].output)

# Image embedding thành vector (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

# # Gọi hàm encode với các ảnh trong traning set
t1 = time.time()

encoding_train = {}
for im in tqdm(train_imgs):
    encoding_train[im[len(images):]] = encode(im)

t2 = time.time()
time_featurize = t2 - t1

result_file = '/content/drive/MyDrive/Inception_LSTM_coco/result.txt'
file = open(result_file,"w")
file.write('time_featurize:\n')
file.write(str(time_featurize))
file.close()

# # # Lưu image embedding lại
with open("/content/drive/MyDrive/Inception_LSTM_coco/encoded_train_images.pkl", "wb") as encoded_pickle:
    dump(encoding_train, encoded_pickle)

train_features = load(open("/content/drive/MyDrive/Inception_LSTM_coco/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))
# print(train_features)

  0%|          | 0/50000 [00:00<?, ?it/s]

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
1/1 [==============================] - 0s 30ms/step


OSError: ignored

In [ ]:
# 3. train_descriptions
train_descriptions = {}
for i in range(0,len(train_imgs)):
    if train_imgs[i] not in train_descriptions:
        annIds = coco.getAnnIds(imgIds = train_ids[i])
        anns = coco.loadAnns(annIds)
        captions = []
        for j in range(0,len(anns)):
            captions.append(anns[j]['caption'].lower())
        train_descriptions[train_imgs[i][:-4]] = captions
# print(train_descriptions)

# Preprocessing text
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(train_descriptions)

# Lưu description xuống file
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(train_descriptions, '/content/drive/MyDrive/Inception_LSTM_coco/descriptions.txt')

# Đọc file các caption
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# Thêm 'startseq', 'endseq' cho chuỗi
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
tr = []
for i in range(len(train_imgs)):
    tr.append(train_imgs[i][:-4])
train = set(tr)
# print('train', train)
train_descriptions = load_clean_descriptions('/content/drive/MyDrive/Inception_LSTM_coco/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# print(train_descriptions)

In [ ]:
# 4. max_length
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description max Length: %d' % max_length)

In [ ]:
# 5. indexing word
# Tạo list các training caption
all_train_captions = []
for key, val in train_descriptions.items():
	for cap in val:
		all_train_captions.append(cap)
len(all_train_captions)

# Chỉ lấy các từ xuất hiện trên 10 lần
word_count_threshold = 3
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1 # Thêm 1 cho từ dùng để padding
print(vocab_size)
# print('wordtoix',wordtoix)
# print('ixtoword',ixtoword)

In [ ]:
# 6. embedding_matrix
# Load Glove model
t1 = time.time()

glove_dir = ''
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, '/content/drive/MyDrive/Inception_LSTM_coco/glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()



print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

t2 = time.time()
time_embedding = t2-t1
file = open(result_file,"a")
file.write('\ntime_embedding:\n')
file.write(str(time_embedding))
file.close()

print('embedding_matrix.shape',embedding_matrix.shape)

In [ ]:
# 7. generator
# data generator cho việc train theo từng batch model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            # photo = photos[key+'.jpg']
            photo = photos[key[len(images):]+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
# 8. model
from tensorflow.keras.utils import plot_model

t1 = time.time()
# Tạo model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

model.summary()
plot_model(model, to_file='InceptionV3_LSTM.png', show_shapes=True)

# Layer 2 dùng GLOVE Model nên set weight thẳng và không cần train
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

# print('train_descriptions',train_descriptions)
# print('train_features',train_features)
# print('wordtoix',wordtoix)
# print('max_length',max_length)
# print('number_pics_per_bath',number_pics_per_bath)

time_epoch = {}

for i in range(epochs):
    t3 = time.time()

    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    
 
    model_hist = model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)

    t4 = time.time()
    time_epoch[i]=t4-t3

model.save('/content/drive/MyDrive/Inception_LSTM_coco/model_30.h5')

t2 = time.time()
time_train = t2 - t1
file = open(result_file,"a")
file.write('\ntime_train\n')
file.write(str(time_train))
file.write('\ntime_epochs\n')
file.write(str(time_epoch))
file.close()

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


#2.val_features
encoding_test = {}
sta = time.time()
for im in tqdm(test_imgs):
    encoding_test[im[len(images):]] = encode(im)

print('extract val_features time:', time.time()-sta)

with open("/content/drive/MyDrive/Inception_LSTM_coco/encoded_test_images.pkl", "wb") as encoded_pickle:
    dump(encoding_test, encoded_pickle)

# print('test_imgs',test_imgs)

test_features = load(open("/content/drive/MyDrive/Inception_LSTM_coco/encoded_test_images.pkl", "rb"))
print('Photos: val=%d' % len(test_features))
# print(val_features['000000511321.jpg'])

# 3. val_descriptions
test_descriptions = {}
for i in range(0,len(test_imgs)):
    if test_imgs[i] not in test_descriptions:
        annIds = coco.getAnnIds(imgIds = test_ids[i])
        anns = coco.loadAnns(annIds)
        captions = []
        for j in range(0,len(anns)):
            captions.append(anns[j]['caption'].lower())
        test_descriptions[test_imgs[i][:-4]] = captions
# print('1 val_descriptions',val_descriptions['000000511321'])

# clean descriptions
clean_descriptions(test_descriptions)
# print('2 val_descriptions', val_descriptions['000000511321'])

save_descriptions(test_descriptions, '/content/drive/MyDrive/Inception_LSTM_coco/descriptions_test.txt')

# descriptions
tr_test = []
for i in range(len(test_imgs)):
    tr_test.append(test_imgs[i][:-4])

test = set(tr_test)
print('test', len(test))
# print(train)
# print(test)
test_descriptions = load_clean_descriptions('/content/drive/MyDrive/Inception_LSTM_coco/descriptions_test.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))

# print(val_descriptions['000000511321'])

# 4. reload tokenizer (wordtoix), recalculate max_length

# 5. evaluate model

def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set

	for key, desc_list in tqdm(descriptions.items()):
		# generate description
		key = str(key)+'.jpg'
		yhat = greedySearch(test_features[key[len(images):]].reshape((1,2048)))
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
		# n += 1
		# if n ==20:
		# 	break
	# calculate BLEU score

	bl1 = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
	bl2 = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
	bl3 = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
	bl4 = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))
 
	print('BLEU-1: %f' % bl1)
	print('BLEU-2: %f' % bl2)
	print('BLEU-3: %f' % bl3)
	print('BLEU-4: %f' % bl4)


	mt = 0
	for i in range(len(predicted)):
		mt += round(meteor_score(actual[i], predicted[i]),4)
	mt = mt/len(predicted) 

	file = open(result_file,"a")
 
	file.write('\nBLEU-1: %f' % bl1)
	file.write('\nBLEU-2: %f' % bl2)
	file.write('\nBLEU-3: %f' % bl3)
	file.write('\nBLEU-4: %f' % bl4)
 
	file.write("\nMETEOR %f" %mt)
	# file.write(str(mt))

	file.close()
from keras.models import load_model

filename = '/content/drive/MyDrive/Inception_LSTM_coco/model_30.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, wordtoix, max_length)

for i in range(10):
	pic = list(test_features.keys())[i]
	image = test_features[pic].reshape((1,2048))
	x=plt.imread(images+str(pic))
	plt.imshow(x)
	plt.show()
	print(greedySearch(image))

In [ ]:
z=20
pic = list(test_features.keys())[z]
image = test_features[pic].reshape((1,2048))
x=plt.imread(images+str(pic))
plt.imshow(x)
plt.show()
print(greedySearch(image))

In [ ]:
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which', 'ensures', 'that', 'the', 'military', 'always', 'obeys', 'the', 'commands', 'of', 'the', 'party']
hypothesis2 = ['It', 'is', 'to', 'insure', 'the', 'troops', 'forever', 'hearing', 'the', 'activity', 'guidebook', 'that', 'party', 'direct']

reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that', 'ensures', 'that', 'the', 'military', 'will', 'forever', 'heed', 'Party', 'commands']
reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which', 'guarantees', 'the', 'military', 'forces', 'always', 'being', 'under', 'the', 'command', 'of', 'the', 'Party']
reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the', 'army', 'always', 'to', 'heed', 'the', 'directions', 'of', 'the', 'party']
ref = [reference1, reference2, reference3]
print(type(ref))
print(type(hypothesis1))

round(meteor_score([reference1, reference2, reference3], hypothesis1),4)